In [1]:
import numpy as np
import pandas as pd
import networkx as nx
from networkx.drawing.nx_agraph import graphviz_layout

In [2]:
data_path = '/notebooks/social network/HW1/data/'
p1 = 'Period1.csv'
p2 = 'Period2.csv'
test = 'TestData.csv'

In [3]:
df_p1 = pd.read_csv(data_path+p1)
df_p1.head()

,year,source id,target id
0,1999,9909133,9811241
1,1997,9702110,9410212
2,1995,9510041,9210121
3,1997,9703036,9611090
4,1997,9610162,9609159


In [4]:
df_p1.shape

(154836, 3)

In [5]:
df_p2 = pd.read_csv(data_path+p2)
df_p2.shape

(98353, 3)

In [6]:
# 隨機取100筆
df1_shuffle = df_p1.sample(n=100)

In [7]:
df1_shuffle.head()

,year,source id,target id
146608,1997,9702038,9210127
79690,1998,9807011,9508068
49190,1996,9612158,9511128
39538,1999,9912266,9808014
56278,1997,9709161,9612207


In [8]:
def to_graph(df):
    G=nx.Graph()
    for i in range(df.shape[0]):
        G.add_edge(df.iloc[i]['source id'],df.iloc[i]['target id'])
        
    return G

In [9]:
%%time
p1_G = to_graph(df1_shuffle)

CPU times: user 24 ms, sys: 0 ns, total: 24 ms
Wall time: 22.6 ms


In [10]:
len(p1_G.edges())

100

In [11]:
p1_nonedge = list(nx.non_edges(p1_G))
len(p1_nonedge)

18428

In [58]:
p1_nonedge[:10]

[(9609219, 9709062),
 (9609219, 9902088),
 (9609219, 9610249),
 (9609219, 9306120),
 (9609219, 9301003),
 (9609219, 9708055),
 (9609219, 9710110),
 (9609219, 9902113),
 (9609219, 9707042),
 (9609219, 9603108)]

In [12]:
# 將P1的non_edge的label設為0
# 並且先做成DF

# create dataframe
label = ['source id','target id']
df_p1_fe = pd.DataFrame.from_records(p1_nonedge, columns=label)
df_p1_fe.head()

,source id,target id
0,9609219,9709062
1,9609219,9902088
2,9609219,9610249
3,9609219,9306120
4,9609219,9301003


In [13]:
def compute_common_neigh(G, nonedge):
    common_neigh = [(e[0], e[1], len(list(nx.common_neighbors(G,e[0],e[1])))) for e in nonedge]
    return common_neigh


def get_list_value(feature_list):
    v = []
    for i in feature_list:
        v.append(i[2])
        
    to_value = pd.Series(v).values
    
    return to_value


def print_output(out):
    print(len(out))
    print(out[:5])

def get_feature(nonedge, G, df):
    common_neigh = compute_common_neigh(G, nonedge)
    v = get_list_value(common_neigh)
    df['common_neigh'] = v
    print_output(common_neigh)
    
    
    jaccard = list(nx.jaccard_coefficient(G))
    v = get_list_value(jaccard)
    df['jaccard'] = v    
    print_output(jaccard)
    
    resource_alloc = list(nx.resource_allocation_index(G))
    v = get_list_value(resource_alloc)
    df['resource_alloc'] = v      
    print_output(resource_alloc)
    
    adamic_adar = list(nx.adamic_adar_index(G))
    v = get_list_value(adamic_adar)
    df['adamic_adar'] = v       
    print_output(adamic_adar)
    
    pref_attach = list(nx.preferential_attachment(G))
    v = get_list_value(pref_attach)
    df['pref_attach'] = v      
    print_output(pref_attach)


In [14]:
# 將P1的 non edge和P1的圖算feature
# 因為要算的是non edge的feature，所以將P1的圖代入API就行
get_feature(p1_nonedge, p1_G, df_p1_fe)

18428
[(9609219, 9709062, 0), (9609219, 9902088, 0), (9609219, 9610249, 0), (9609219, 9306120, 0), (9609219, 9301003, 0)]
18428
[(9609219, 9709062, 0.0), (9609219, 9902088, 0.0), (9609219, 9610249, 0.0), (9609219, 9306120, 0.0), (9609219, 9301003, 0.0)]
18428
[(9609219, 9709062, 0), (9609219, 9902088, 0), (9609219, 9610249, 0), (9609219, 9306120, 0), (9609219, 9301003, 0)]
18428
[(9609219, 9709062, 0), (9609219, 9902088, 0), (9609219, 9610249, 0), (9609219, 9306120, 0), (9609219, 9301003, 0)]
18428
[(9609219, 9709062, 1), (9609219, 9902088, 1), (9609219, 9610249, 1), (9609219, 9306120, 1), (9609219, 9301003, 1)]


In [15]:
df_p1_fe.head()

,source id,target id,common_neigh,jaccard,resource_alloc,adamic_adar,pref_attach
0,9609219,9709062,0,0.0,0.0,0.0,1
1,9609219,9902088,0,0.0,0.0,0.0,1
2,9609219,9610249,0,0.0,0.0,0.0,1
3,9609219,9306120,0,0.0,0.0,0.0,1
4,9609219,9301003,0,0.0,0.0,0.0,1


In [16]:
df_p1_fe['label'] = 0
df_p1_fe.head()

,source id,target id,common_neigh,jaccard,resource_alloc,adamic_adar,pref_attach,label
0,9609219,9709062,0,0.0,0.0,0.0,1,0
1,9609219,9902088,0,0.0,0.0,0.0,1,0
2,9609219,9610249,0,0.0,0.0,0.0,1,0
3,9609219,9306120,0,0.0,0.0,0.0,1,0
4,9609219,9301003,0,0.0,0.0,0.0,1,0


In [47]:
df_p1_fe.shape

(18428, 8)

In [40]:
# P2隨機取10000筆，讓lable比重不要差太多，也不至於算太久
# P2在這裡會被當成是label全為1的資料
df2_shuffle = df_p2.sample(n=10000)
p2_G = to_graph(df2_shuffle)

In [41]:
len(p2_G.edges())

9998

In [42]:
# P2取補圖是為了要在算feature時可以算回來當初取的那10000筆資料的feature
com_p2 = nx.complement(p2_G)
len(com_p2.edges())

28563022

In [43]:
p2_nonedge = list(nx.non_edges(com_p2))
len(p2_nonedge)

9998

In [44]:
# 將那10000筆資料建成DF，label會是1

# create dataframe
label = ['source id','target id']
df_p2_fe = pd.DataFrame.from_records(p2_nonedge, columns=label)
df_p2_fe.head()

,source id,target id
0,9601024,6005
1,9601028,110076
2,9601029,103171
3,9601029,2116
4,9601029,104198


In [45]:
df_p2_fe.shape

(9998, 2)

In [48]:
# 利用補圖可以算回來當初那10000筆資料的feature
get_feature(p2_nonedge, com_p2, df_p2_fe)

9998
[(9601024, 6005, 7558), (9601028, 110076, 7557), (9601029, 103171, 7536), (9601029, 2116, 7541), (9601029, 104198, 7544)]
9998
[(9601024, 6005, 1.0), (9601028, 110076, 0.9998676898650437), (9601029, 103171, 0.9970891770309606), (9601029, 2116, 0.9977507277057422), (9601029, 104198, 0.9981476581106112)]
9998
[(9601024, 6005, 1.0002179212243434), (9601028, 110076, 1.0000855585598876), (9601029, 103171, 0.9973047832377225), (9601029, 2116, 0.9979673347369635), (9601029, 104198, 0.9983642825433556)]
9998
[(9601024, 6005, 846.3469296506662), (9601028, 110076, 846.2349471278646), (9601029, 103171, 843.8832044225545), (9601029, 2116, 844.4431868854705), (9601029, 104198, 844.7791211735424)]
9998
[(9601024, 6005, 57123364), (9601028, 110076, 57115806), (9601029, 103171, 56957208), (9601029, 2116, 56994938), (9601029, 104198, 57017576)]


In [49]:
df_p2_fe.head()

,source id,target id,common_neigh,jaccard,resource_alloc,adamic_adar,pref_attach
0,9601024,6005,7558,1.000000,1.000218,846.346930,57123364
1,9601028,110076,7557,0.999868,1.000086,846.234947,57115806
2,9601029,103171,7536,0.997089,0.997305,843.883204,56957208
3,9601029,2116,7541,0.997751,0.997967,844.443187,56994938
4,9601029,104198,7544,0.998148,0.998364,844.779121,57017576


In [50]:
df_p2_fe.shape

(9998, 7)

In [51]:
df_p2_fe['label'] = 1
print(df_p2_fe.shape)
df_p2_fe.head()

(9998, 8)


,source id,target id,common_neigh,jaccard,resource_alloc,adamic_adar,pref_attach,label
0,9601024,6005,7558,1.000000,1.000218,846.346930,57123364,1
1,9601028,110076,7557,0.999868,1.000086,846.234947,57115806,1
2,9601029,103171,7536,0.997089,0.997305,843.883204,56957208,1
3,9601029,2116,7541,0.997751,0.997967,844.443187,56994938,1
4,9601029,104198,7544,0.998148,0.998364,844.779121,57017576,1


In [52]:
all_df = pd.concat([df_p2_fe, df_p1_fe], axis=0)
all_df.head()

,source id,target id,common_neigh,jaccard,resource_alloc,adamic_adar,pref_attach,label
0,9601024,6005,7558,1.000000,1.000218,846.346930,57123364,1
1,9601028,110076,7557,0.999868,1.000086,846.234947,57115806,1
2,9601029,103171,7536,0.997089,0.997305,843.883204,56957208,1
3,9601029,2116,7541,0.997751,0.997967,844.443187,56994938,1
4,9601029,104198,7544,0.998148,0.998364,844.779121,57017576,1


In [53]:
all_df.shape

(28426, 8)

In [54]:
# 檢查是否label 1的邊和label 0的邊重複

not_duplicated_row = all_df.duplicated(['source id', 'target id'])
not_duplicated_row.value_counts()

False    28426
dtype: int64

In [55]:
# 如果有重複就把label 0的邊刪掉
# 因為這代表那個邊在未來會出現，不該為0
# 然後將DF洗亂，方便去TRAIN

all_df = all_df[~not_duplicated_row].sample(frac=1).reset_index(drop=True)
all_df.head(10)

,source id,target id,common_neigh,jaccard,resource_alloc,adamic_adar,pref_attach,label
0,9802067,9910135,0,0.000000,0.000000,0.000000,1,0
1,9406055,9910067,0,0.000000,0.000000,0.000000,1,0
2,103230,112191,7555,0.999603,0.999821,846.010967,57100690,1
3,9704098,9412062,0,0.000000,0.000000,0.000000,1,0
4,9708149,9412062,0,0.000000,0.000000,0.000000,1,0
5,9909003,9912266,0,0.000000,0.000000,0.000000,1,0
6,9808057,9704184,0,0.000000,0.000000,0.000000,1,0
7,9302103,9410167,0,0.000000,0.000000,0.000000,1,0
8,2190,104067,7548,0.998677,0.998894,845.227076,57047793,1
9,9508021,9802163,0,0.000000,0.000000,0.000000,1,0


In [56]:
# 檢查target重複的項目與次數
all_df['target id'].value_counts()[:20]

9910263    192
9410046    191
9608186    190
9611230    189
9903093    188
9601010    187
9308139    186
9803235    186
9411048    185
9910238    184
9909205    183
9412062    181
9612251    180
9902043    179
9902040    179
9612246    177
9309140    176
9902035    174
9604051    173
9710032    172
Name: target id, dtype: int64

In [57]:
%%time
all_df.to_csv('process1.csv', index=False)

CPU times: user 332 ms, sys: 12 ms, total: 344 ms
Wall time: 339 ms


In [59]:
all_df['target id'].value_counts()

9910263    192
9410046    191
9608186    190
9611230    189
9903093    188
9601010    187
9308139    186
9803235    186
9411048    185
9910238    184
9909205    183
9412062    181
9612251    180
9902043    179
9902040    179
9612246    177
9309140    176
9902035    174
9604051    173
9710032    172
9802150    172
9912266    171
9305040    171
9604041    169
9503176    168
9307079    168
9606086    167
9611203    166
9902019    165
9605054    162
          ... 
9903189      1
9712239      1
105082       1
9704059      1
105210       1
9608074      1
7034         1
9909103      1
7002         1
101176       1
9612076      1
11036        1
103195       1
105242       1
11020        1
11004        1
101112       1
109180       1
9403126      1
9908114      1
101064       1
107197       1
103099       1
105146       1
8166         1
103083       1
111263       1
9705124      1
9704075      1
4098         1
Name: target id, Length: 5137, dtype: int64